ТЗ:

Сотрудники института биоинформатики хотят восстановить первичную структуру белка, для чего требуется объединить его (частично пересекающиеся) фрагменты. Требуется написать программу, которая хранит белковые последовательности (в виде списков аминокислот) и умеет их объединять, если аминокислоты в конце одной последовательности совпадут с аминокислотами в начале другой.

1.1.	Класс трансляции протеиногенной аминокислоты, умеющий синтезировать ее по хранимому коду из последовательно поступающих нуклеотидов, закодированных при помощи однобуквенных обозначений.

1.2.	Класс белковой последовательности, умеющий синтезировать последовательность из нуклеотидов и других аминокислот. 

1.3.	Белок (последовательность аминокислот); операция сложения, возвращающая результат сложения (с пересечением) двух белков


In [2]:
from dataclasses import dataclass

# Нуклеотиды

В протеиногенных аминокислотах участвуют 4 главных нуклеотида. Все они обозначаются одной буквой и имеют какое-то название.
Отразим это в следующем словаре:

In [3]:
nucleotide_dict = {
    'A': 'Аденин',
    'G': 'Гуанин',
    'C': 'Цитозин',
    'U': 'Урацил'
}

Теперь создаём класс нуклеотида. 
Пока он будет хранить только своё буквенное обозначение и название. Также он будет уметь выводить обе эти позиции в методе display(). Возможность знать, что обозначает та или иная буква, коллегам точно не помешает. 

*Если коллегам потребуется добавить, например, молекулярную структуру нуклеотида, мы создадим соответсвующий класс, и при инициализации нуклеотид будет добавлять себе его переменную. Но пока ограничимся буквой и названием.*

In [4]:
class nucleotide:
    def __init__(self, code: str): # Объект инициализируется по букве нуклеотида
            
            self.code = code
            self.name = ''
            
            try:
                self.name = nucleotide_dict[code] # Конструктор ищет название нуклеотида в словаре по введенной букве
            except:
                raise Exception('Non-existent nucleotide code: ' + code)
                # Если пользователь дал конструктору что-то кроме буквы нуклеотида, мы останавливаем его
            
            self.display_str = self.code + ' | ' + self.name # В эту строчку нуклеотид записывает всё, что он знает о себе
    
    def display(self): # Этим методом он рассказывает всё, что он знает о себе
        print(self.display_str)
               

Попробуем создать два нуклеотида и вывести информацию о них:

In [5]:
A = nucleotide("A")
A.display()

G = nucleotide("G")
G.display()

A | Аденин
G | Гуанин


Теперь попробуем всё сломать:

In [9]:
banana = nucleotide('Banana') # Попытаемся запихать в конструктор банан
banana.display()

Exception: Non-existent nucleotide code: Banana

# Аминокислоты

Помимо названия и буквенного обозначения, каждая протеиногенная аминокислота также имеет от одной до шести комбинаций нуклеотидов, в которых она может быть выражена. Отразим это в специальном списке аминокислот, каждый элемнт в котором будет хранить имя, возможные комбинации и букву. 

In [10]:
@dataclass() # Класс для элементов списка
class am_ac_entry():
    name: str
    nucs: [str]
    letter: str
        
        
#Взяты отсюда: https://ru.wikipedia.org/wiki/Аминокислоты#Протеиногенные_аминокислоты
amino_acid_list = [
    am_ac_entry('Фенилаланин', ['UUU','UUC'], 'F'),
    am_ac_entry('Лейцин', ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], 'L'),
    am_ac_entry('Цистеин', ['UGU', 'UGC'], 'C'),
    am_ac_entry('Триптофан', ['UGG'], 'W'),
    am_ac_entry('Серин', ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'S'),
    am_ac_entry('Тирозин', ['UAU', 'UAC'], 'Y'),
    am_ac_entry('Валин', ['GUU', 'GUC', 'GUA', 'GUG'], 'V'),
    am_ac_entry('Глицин', ['GGU', 'GGC', 'GGA', 'GGG'], 'G'),
    am_ac_entry('Аланин', ['GCU', 'GCC', 'GCA', 'GCG'], 'A'),
    am_ac_entry('Аспарагиновая кислота', ['GAU', 'GAC'], 'D'),
    am_ac_entry('Глутаминовая кислота', ['GAA', 'GAG'], 'E'),
    am_ac_entry('Аргинин', ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'R'),
    am_ac_entry('Пролин', ['CCU', 'CCC', 'CCA', 'CCG'], 'P'),
    am_ac_entry('Гистидин', ['CAU', 'CAC'], 'H'),
    am_ac_entry('Глутамин', ['CAA', 'CAG'], 'Q'),
    am_ac_entry('Изолейцин', ['AUU', 'AUC', 'AUA'], 'I'),
    am_ac_entry('Метионин', ['AUG'], 'M'),
    am_ac_entry('Треонин', ['ACU', 'ACC', 'ACA', 'ACG'], 'T'),
    am_ac_entry('Аспарагин', ['AAU', 'AAC'], 'N'),
    am_ac_entry('Лизин', ['AAA', 'AAG'], 'K'),
    
]

Создадим функцию, которая будет возвращать конструктору свойства аминокислоты из списка.
По умолчанию на вход она принимает три символа (str), находит соответствие с комбинациями нуклеотидов и возвращет название кислоты и её буквенное обозначение, если такое есть в списке.

Также функция может искать название и комбинацию нуклеотидов по буквенному обозначению кислоты. В таком случае by_latter необходмо перевести в положение True.

In [11]:
def amino_name_define(nucl_combo: str, by_latter = False):
    if by_latter:
        for entry in amino_acid_list:
            if entry.letter == nucl_combo:
                return entry.name, entry.nucs[0], entry.letter
            # При поиске по букве кислоты возвращается первая возможная комбинация нуклеотидов
    else:
        for entry in amino_acid_list:
            for nucls in entry.nucs:
                if nucls == nucl_combo:
                    return entry.name, entry.letter

Теперь создаём класс аминокислоты. 
Он будет хранить в себе список входящих в кислоту нуклеотидов, её название и буквенное обозначенине. 

In [12]:
class amino_acid(nucleotide): # От нуклеотида мы наследуем метод display()
    
    def __init__(self, nucleotides: [nucleotide] = [], letter: str = None):
        
        self.nulceotides = []
        self.name = ''
        self.letter = ''
        
        
        # По умолчанию __init__ создает аминокислоту на основе списка переданных нуклеотидов
        if len(nucleotides) > 1:

            self.nucleotides = nucleotides
            self.code = '' #код комбинации нуклеотидов (AAA, CGU и т.д.) в виде строки
            self.code = self.code.join([self.code + nucleo.code for nucleo in nucleotides])
            
            # Если число нуклеотидов не соответсвует 3 то amino_name_define не находит кислоту, вызывается исключение 
            try:
                self.name, self.letter = amino_name_define(self.code)
            except:
                raise Exception('An error occured while trying to initialize the amino acid! Unacceptable nucleotides input: ' + self.code)
                
        # Если пользователь не хочет сначала инициализировать каждый нуклеотид, чтобы потом собрать из них кислоту,
        # он может сразу создать кислоту при помощи её обозначения в одну букву.
        # Например, если мы хотим создать аргинин, мы можем сделать это так: 
        # arginin = amino_acid([], 'R')
        # В таком случе будет взята первая возможная комбинация нуклеотидов из amino_acid_list - то есть CGU
        
        else:
            # Тогда программа проверяет, есть ли искомая буква в списке аминокислот
            try:
                self.name, self.code, self.letter = amino_name_define(letter, by_latter = True)
                self.nucleotides = [nucleotide(letter) for letter in self.code]
            # И бросает исключение, если не находит
            except:
                raise Exception('An error occured while trying to initialize the amino acid! Unacceptable letter input: ' + letter)
               
        self.display_str = self.code + ' | ' + self.letter +' | ' + self.name # это используется функцией display() от нуклеотида
           


Попробуем создать аминокислоты всеми доступными способами:

In [13]:
# Из существующих нуклеотидов
acid = amino_acid([A,G,G])
acid.display()

# С созданием нуклеотидов прямо при инициализации
acid2 = amino_acid([A, nucleotide('C'), nucleotide('U')])
acid2.display()

# При помощи буквенного обозначение кислоты
acid3 = amino_acid([],'W')
acid3.display()

AGG | R | Аргинин
ACU | T | Треонин
UGG | W | Триптофан


Теперь опять попробуем дать неправлильный ввод:

In [14]:
# Число нуклеотидов больше 3
acid_error3 = amino_acid([A,A,A,A,G,G,G,G])

Exception: An error occured while trying to initialize the amino acid! Unacceptable nucleotides input: AAAAGGGG

In [15]:
# Попросим у конструктора банановую кислоту
acid_error2 = amino_acid([], 'banana acid')

Exception: An error occured while trying to initialize the amino acid! Unacceptable letter input: banana acid

# Белковая последовательность

Класс белковой последовательности хранит в себе список аминокислот, а также список их обозначений в одну букву (в виде строки). 
Он создаёт список аминокислот при инициализации. Это можно сделать при помощи отдельных нуклеотидов, уже готовых аминокислот или просто при помощи списка буквенных обозначений кислот.

Также можно комбинировать подходы, например, мы даём конструктору набор нуклеотидов AGG (мы не помним, что это за аминокислота) и просим его присоединить к нему кислоты QHF. Он узнает в нуклеотидах аргинин (R) и создаёт нам последовательность RQHF.

In [19]:
ch = protein_chain([A,G,G],[],'QHF')
ch.display()

RQHF


Также класс белковой последовательности получает метод full_display(), который показывает не только список кислот в последовательности, но и расписывает каждую кислоту с названием, буквенным обозначением и списком нуклеотидов.

In [31]:
class protein_chain(nucleotide): # Снова берём метод display()
    
    def __init__(self, nucs: [nucleotide] = [], am_acids: [amino_acid] = [],  init_str: str = ''):
        
        self.prot_chain = []
        str_prot_chain = []
        self.prot_chain_code = ''
        
        if init_str != '': 
            for acid_letter in init_str:
                str_prot_chain.append(amino_acid([], acid_letter)) # Создаём из строки букв список аминокислот
        
        # Проверяется количество переданных в конструктор нуклеотидов
        # Протеиногенные аминокислоты состоят из 3 нуклеотидов, поэтому если их общее число не кратно 3, 
        # Цепь не будет создана    
        if len(nucs) % 3 == 0:
            self.nuc_triplets = []
            i = 0
            j = 0
            max_i = len(nucs) / 3
            while (i < max_i):
                nuc_triplet = nucs[j:j+3] # Создаём тройки из нуклеотидов
                self.nuc_triplets.append(nuc_triplet)
                j += 3
                i += 1
            self.new_acids = []    
            for triplet in self.nuc_triplets:
                self.new_acids.append(amino_acid(triplet)) # Превращаем тройки в кислоты и добавляем к списку
            self.prot_chain = self.new_acids + am_acids + str_prot_chain
            
            # Создаем строку буквенных обозначений кислот
            self.prot_chain_code = self.prot_chain_code.join([self.prot_chain_code + acid.letter for acid in self.prot_chain])

            
            self.display_str = self.prot_chain_code
        else:
            raise Exception('Unacceptable number of nucleotides. Protein chain was not creatred. Number of nucleotides must be divisible by 3.')
    
    #Обычый display() показывает только название цепочки
    def full_display(self): #Эта функция показывает название цепочки, а также названия всех аминокислот, входящих в неё
        self.display()
        
        for acid in self.prot_chain:
            acid.display() 
            

Попробуем создать белковые последовательности.

In [32]:
# Нуклеотидами и кислотами + сравниваем методы display() и full_display()
chain = protein_chain([A,A,A,G,G,G,A,A,G],[acid, acid2, acid3])
chain.display()
print('------')
chain.full_display() 

KGKRTW
------
KGKRTW
AAA | K | Лизин
GGG | G | Глицин
AAG | K | Лизин
AGG | R | Аргинин
ACU | T | Треонин
UGG | W | Триптофан


In [33]:
# Только нуклеотидами
U = nucleotide('U')
C = nucleotide('C')

chain2 = protein_chain([A,C,U,A,A,A,U,U,U,G,G,C,C,U,C,A,A,C],[])
chain2.full_display()

TKFGLN
ACU | T | Треонин
AAA | K | Лизин
UUU | F | Фенилаланин
GGC | G | Глицин
CUC | L | Лейцин
AAC | N | Аспарагин


In [34]:
# Только аминокислотами
acid4 = amino_acid([],'I')
acid5 = amino_acid([],'Q')
acid6 = amino_acid([], 'Y')

chain3 = protein_chain([],[acid4, acid2, acid5, acid, acid6, acid3, acid4])
chain3.full_display()

ITQRYWI
AUU | I | Изолейцин
ACU | T | Треонин
CAA | Q | Глутамин
AGG | R | Аргинин
UAU | Y | Тирозин
UGG | W | Триптофан
AUU | I | Изолейцин


In [38]:
# Только при помощи буквенных обозначений
acid_by_letters = protein_chain([],[], 'DERP')
acid_by_letters.full_display()

DERP
GAU | D | Аспарагиновая кислота
GAA | E | Глутаминовая кислота
CGU | R | Аргинин
CCU | P | Пролин


In [39]:
# Всеми способами сразу
acid_combined_init = protein_chain([G,A,U,C,C,A,U,U,C],[acid4, acid5, acid6],'FFRPIQ')
acid_combined_init.full_display()

DPFIQYFFRPIQ
GAU | D | Аспарагиновая кислота
CCA | P | Пролин
UUC | F | Фенилаланин
AUU | I | Изолейцин
CAA | Q | Глутамин
UAU | Y | Тирозин
UUU | F | Фенилаланин
UUU | F | Фенилаланин
CGU | R | Аргинин
CCU | P | Пролин
AUU | I | Изолейцин
CAA | Q | Глутамин


Теперь снова попробуем всё сломать.

In [40]:
# Дадим число нуклеотидов, не кратное 3
err_prot_chain1 = protein_chain([A,C,U,A],[])

Exception: Unacceptable number of nucleotides. Protein chain was not creatred. Number of nucleotides must be divisible by 3.

In [42]:
# Дадим непонятную строку
some_str = 'rsff345er'
err_prot_chain2 = protein_chain([],[],some_str)

Exception: An error occured while trying to initialize the amino acid! Unacceptable letter input: r

# Белок

Класс белка, по сути, повторяет функционал, способы инициализации белковой последовательности.
Главное дополнение к нему - оператор "+", который теперь позволяет получать новые белки путём сложения аминокислот, входящих в них.

Перед сложением внутренний метод check_end_acids() проверяет, совпадают ли кислоты на концах двух последовательностей. Если да, то он возвращает новый белок. 

Также к белку можно прибавить список аминокислот сразу, без создания отдельного белка или белковой последовательности - ради экономии времени.

In [44]:
class protein(protein_chain):
    def __add__(self, other):
        if isinstance(other, protein) or isinstance(other, protein_chain):
            return self.check_end_acids(self, other)
        
        elif type(other) == str:
            try:
                prot_to_add = protein_chain([],[],other)
            except:
                raise Exception("Unable to add proteins: string value has to consist only of amino acid one letter codes.")
            return self.check_end_acids(self, prot_to_add)
            
        else:
            raise Exception("Value to be added to protein must be a protein, a protein_chain or a string of amino acids!")
            
    def check_end_acids(self, protein1, protein2):
        if protein1.prot_chain[-1].letter == protein2.prot_chain[0].letter:
            return protein([],protein1.prot_chain + protein2.prot_chain)
        elif protein2.prot_chain[-1].letter == protein1.prot_chain[0].letter:
            return protein([],protein2.prot_chain + protein1.prot_chain)
        else:
            raise Exception("Unable to add proteins: end amino acids do not match.")

Создаём белки.

In [45]:
I = amino_acid([], 'I')
F = amino_acid([], 'F')
H = amino_acid([], 'H')
Q = amino_acid([], 'Q')
M = amino_acid([], 'M')


protein1 = protein([],[F,F,H,H,I])
protein1.display()

protein2 = protein([],[I,H,I,I,F,F,H])
protein2.display()

FFHHI
IHIIFFH


Складываем белки.

In [51]:
protein3 = protein1 + protein2
protein3.display()

FFHHIIHIIFFH


In [50]:
p_chain = protein_chain([],[Q,H,M,F])
p_chain.display()

# Мы также можем складывать белки с белковыми последовательностями
protein4 = protein3 + p_chain
protein4.display()

QHMF
QHMFFFHHIIHIIFFH


Пробуем ложить белок с последовательностью в виде строки.

In [47]:
new_protein = protein([],[],'IFF')
new_protein2 = new_protein + 'FHQK'
new_protein2.display()

IFFFHQK


В любой момент времени мы можем обратиться к любой из аимнокислот в белке и посмотреть, из каких нуклеотидов она состоит.

In [52]:
new_protein2.prot_chain[0].display()

for nuc in new_protein2.prot_chain[0].nucleotides:
    nuc.display()

AUU | I | Изолейцин
A | Аденин
U | Урацил
U | Урацил


Функция full_display(), унаследованная от protein_chain, работает и отражает всё, что хранится в белке.

In [53]:
protein4.full_display()

QHMFFFHHIIHIIFFH
CAA | Q | Глутамин
CAU | H | Гистидин
AUG | M | Метионин
UUU | F | Фенилаланин
UUU | F | Фенилаланин
UUU | F | Фенилаланин
CAU | H | Гистидин
CAU | H | Гистидин
AUU | I | Изолейцин
AUU | I | Изолейцин
CAU | H | Гистидин
AUU | I | Изолейцин
AUU | I | Изолейцин
UUU | F | Фенилаланин
UUU | F | Фенилаланин
CAU | H | Гистидин


Теперь снова попробуем всё сломать.
Попытаемся сложить белки, конечные кислоты которых не совпадают. 

In [54]:
err_protein1 = protein2 + protein4

Exception: Unable to add proteins: end amino acids do not match.

Теперь попробуем то же самое со строкой, содержащей ошибочный ввод кислот:

In [55]:
err_protein2 = protein2 + 'banana'

Exception: Unable to add proteins: string value has to consist only of amino acid one letter codes.

# Заключение

Теперь, если коллеги захотят создать, например, белок FQHDPIRRKGKTW, то они могут:

In [56]:
# Создать нужные им нуклеотиды
nuc_a = nucleotide('A')
nuc_c = nucleotide('C')
nuc_g = nucleotide('G')
nuc_u = nucleotide('U')

In [58]:
# Собрать из них несколько кислот
acid_f = amino_acid([nuc_u,nuc_u,nuc_u])
acid_f.display()

acid_q = amino_acid([nuc_c,nuc_a,nuc_a])
acid_q.display()

UUU | F | Фенилаланин
CAA | Q | Глутамин


In [62]:
# Понять, что это очень муторный процесс, и вообще можно создавать кислоты быстрее - буквами
acid_h = amino_acid([],'H')
acid_h.display()
acid_d = amino_acid([],'D')
acid_d.display()

CAU | H | Гистидин
GAU | D | Аспарагиновая кислота


In [63]:
# Понять, что и это требует очень много времени, и на самом деле уже можно создать цепочку при помощи уже существующих кислот
# и буквенных обозначений новых 
protein_ch1 = protein_chain([],[acid_f, acid_q, acid_h, acid_d], 'PIR')
protein_ch1.display()

FQHDPIR


In [64]:
# И вообще целый белок можно создать только бувенными кодами
_protein = protein([],[],'RKGKTW')
_protein.display()

RKGKTW


In [65]:
# И теперь белок с цепочкой можно сложить и получить то, что нам нужно было
needed_protein = _protein + protein_ch1
needed_protein.full_display()

FQHDPIRRKGKTW
UUU | F | Фенилаланин
CAA | Q | Глутамин
CAU | H | Гистидин
GAU | D | Аспарагиновая кислота
CCU | P | Пролин
AUU | I | Изолейцин
CGU | R | Аргинин
CGU | R | Аргинин
AAA | K | Лизин
GGU | G | Глицин
AAA | K | Лизин
ACU | T | Треонин
UGG | W | Триптофан
